In [1451]:
import numpy as np
import os
import pandas as pd
import math
from sklearn.model_selection import train_test_split

In [1452]:
#Importing the data
os.chdir("C:/Users/Joey/Desktop/Pima")
os.listdir()
data=pd.read_csv("pima-indians-diabetes.csv")
#Storing accuracy for 10 iterations
accuracyOfList=[]

In [1621]:
data


Number of pregnancies  Glucose concentration  Blood Pressure  \
0                        6                    148              72   
1                        1                     85              66   
2                        8                    183              64   
3                        1                     89              66   
4                        0                    137              40   
..                     ...                    ...             ...   
763                     10                    101              76   
764                      2                    122              70   
765                      5                    121              72   
766                      1                    126              60   
767                      1                     93              70   

     Skin thickness  Insulin  Body mass index  Diabetes pedigree function  \
0                35        0             33.6                       0.627   
1                29        0             26.6                       0.351   
2                 0        0             23.3                       0.672   
3                23       94             28.1                       0.167   
4                35      168             43.1                       2.288   
..              ...      ...              ...                         ...   
763              48      180             32.9                       0.171   
764              27        0             36.8                       0.340   
765              23      112             26.2                       0.245   
766               0        0             30.1                       0.349   
767              31        0             30.4                       0.315   

     Age  Outcome  
0     50        1  
1     31        0  
2     32        1  
3     21        0  
4     33        1  
..   ...      ...  
763   63        0  
764   27        0  
765   30        0  
766   47        1  
767   23        0  

[768 rows x 9 columns]

In [1622]:
# have taken features for training 
features=["Glucose concentration","Blood Pressure","Skin thickness","Outcome"]
#A data frame is created with the list features
X_frame=data[features]
#The data-set has been splitted into two (Training and testing)
X_train,X_test=train_test_split(X_frame, test_size=0.5)
#Defining two data frames for two classes A("Class Variable==0") and B("Class Variable==1")
X_trainA=X_train[X_train["Outcome"]==0]
X_trainB=X_train[X_train["Outcome"]==1]

In [1623]:
#Calculating prior probablities of the classes
prior_probability_A=X_trainA.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])
prior_probability_B=X_trainB.shape[0]/(X_trainA.shape[0]+X_trainB.shape[0])

In [1624]:
y_trainA=X_trainA[["Outcome"]]
y_trainB=X_trainB[["Outcome"]]

In [1625]:
#Removing "Outcome" feature from both the dataframes
X_trainA=X_trainA[["Glucose concentration","Blood Pressure","Skin thickness"]]
X_trainB=X_trainB[["Glucose concentration","Blood Pressure","Skin thickness"]]

In [1626]:
#function for calculating the mean
def mean(x):
    return sum(x)/x.shape[0]

In [1627]:
#mean of the features of classA
meanA1=mean(X_trainA.iloc[:,0])
meanA2=mean(X_trainA.iloc[:,1])
meanA3=mean(X_trainA.iloc[:,2])

In [1628]:
#mean of the features of classB
meanB1=mean(X_trainB.iloc[:,0])
meanB2=mean(X_trainB.iloc[:,1])
meanB3=mean(X_trainB.iloc[:,2])

In [1629]:
#function which takes mean and covariance as the parameters and returns the likelihood of the feature vector
def likelihood(x,mean,cov):
    #inverse of the covariance matrix
    inverse=np.linalg.inv(cov)
    a1=1/(np.sqrt(((2*np.pi)**3)*np.linalg.det(cov)))
    a2=np.exp(-0.5*np.dot(np.dot((x-mean).T,inverse),(x-mean)))
    a=a1*a2
    return a
    

In [1630]:
#defining the mean which stacked all the means of the taken features
MeanVecA=np.array([[meanA1,meanA2,meanA3]])
MeanVecB=np.array([[meanB1,meanB2,meanB3]])

In [1631]:
#calculating the covariance matrix for both classes(A and B)
covarianceA=np.cov(X_trainA.T)
covarianceB=np.cov(X_trainB.T)

In [1632]:
#Testing one test sample on the classifier
p=X_test.iloc[381,0:3].to_numpy().reshape(X_test.shape[1]-1,1)
posA=likelihood(p,MeanVecA.T,covarianceA)*prior_probability_A
posB=likelihood(p,MeanVecB.T,covarianceB)*prior_probability_B
if(posA<posB):
    print("Class B")
else:
    print("Class A")

Class A


In [1633]:
X_test.iloc[381,3]


0

In [1634]:
#Accuracy of the test set
predicted_res=[]
for k in range(X_test.shape[0]):
    #changing the test sample from pandas series to numpy array and reshaping it
    p=X_test.iloc[k,0:3].to_numpy().reshape(X_test.shape[1]-1,1)
    #calculating posterior probabilites
    posA=likelihood(p,MeanVecA.T,covarianceA)*prior_probability_A
    posB=likelihood(p,MeanVecB.T,covarianceB)*prior_probability_B
    #whichever posterior probability is more,the test sample is labelled with the class label
    if(posA<posB):
        predicted_res.append(1)
    else:
        predicted_res.append(0)
XTest=X_test.copy()
XTest["predictedRes"]=predicted_res
correct=0
incorrect=0
#Checking how many did the classifier correctly labelled
for k in range(XTest.shape[0]):
    if(XTest.iloc[k,3]==XTest.iloc[k,4]):
        correct+=1
    else:
        incorrect+=1
        
print("Total number of correctly predicted values:"+str(correct))
accuracy=correct/X_test.shape[0]
print("Accuracy:"+str(accuracy))
print("Total number of incorrectly predicted values:"+str(wrong))


Total number of correctly predicted values:283
Accuracy:0.7369791666666666
Total number of incorrectly predicted values:90


In [1635]:
accuracyOfList.append(accuracy)

In [1636]:
accuracyOfList

[0.78125,
 0.734375,
 0.734375,
 0.7604166666666666,
 0.71875,
 0.71875,
 0.75,
 0.7291666666666666,
 0.765625,
 0.7369791666666666]

In [1640]:
mean=sum(accuracyOfList)/len(accuracyOfList)
print(mean)

0.74296875


In [1641]:
import math
t=0
for a in range(len(accuracyOfList)):
 t+=(accuracyOfList[a]-mean)**2
s=t/(len(accuracyOfList)-1)
std_deviation=math.sqrt(s)


In [1642]:
#standard deviation
print(std_deviation)

0.020762684027659028
